<div style="display: flex; background-color: #3F579F;">
    <h1 style="margin: auto; padding: 30px 30px 0px 30px;">Design an application for public health - Project 3</h1>
</div>
<div style="display: flex; background-color: #3F579F; margin: auto; padding: 5px 30px 0px 30px;">
    <span style="width: 100%; text-align: center; font-size:20px; font-weight: bold; float: left;">| Cleaning notebook |</span>
</div>
<div style="display: flex; background-color: #3F579F; margin: auto; padding: 10px 30px 30px 30px;">
    <span style="width: 100%; text-align: center; font-size:26px; float: left;">Data Scientist course - OpenClassrooms</span>
</div>

<div style="background-color: #506AB9;" >
    <h2 style="margin: auto; padding: 20px; color:#fff; ">1. Libraries and functions</h2>
</div>

<div style="background-color: #6D83C5;" >
    <h3 style="margin: auto; padding: 20px; color:#fff; ">1.1. Libraries and functions</h3>
</div>

In [1]:
import io
import gc
from math import prod
import time as time
import pandas as pd

<div class="alert alert-block alert-info">
Due to size of the dataset, it is necessary to show all columns to work on it
</div>

In [2]:
pd.set_option("display.max_columns", None) # show max of cols
pd.set_option("max_colwidth", None) # show full width of cols

<div style="background-color: #6D83C5;" >
    <h3 style="margin: auto; padding: 20px; color:#fff; ">1.2. Functions declaration</h3>
</div>

In [3]:
def df_initial_analysis(df, name_df):
    """
    Initial analysis on the DataFrame.

    Args:
        df (pandas.DataFrame): DataFrame to analyze.
        name_df (str): DataFrame name.

    Returns:
        None. 
        Print the initial analysis on the DataFrame. 
    """
    
    # Calculating the memory usage based on dataframe.info()
    buf = io.StringIO()
    df.info(buf=buf)
    memory_usage = buf.getvalue().split('\n')[-2]
  
    if df.empty:
        print("The", name_df, "dataset is empty. Please verify the file.")
    else:
        empty_cols = [col for col in df.columns if df[col].isna().all()] # identifying empty columns
        df_rows_duplicates = df[df.duplicated()] #identifying full duplicates rows
        
        # Creating a dataset based on Type object and records by columns
        type_cols = df.dtypes.apply(lambda x: x.name).to_dict() 
        df_resume = pd.DataFrame(list(type_cols.items()), columns = ["Name", "Type"])
        df_resume["Records"] = list(df.count())
        
        print("\nInitial Analysis of", name_df, "dataset")
        print("--------------------------------------------------------------------------")
        print("- Dataset shape:                 ", df.shape[0], "rows and", df.shape[1], "columns")
        print("- Total of NaN values:           ", df.isna().sum().sum())
        print("- Percentage of NaN:             ", round((df.isna().sum().sum() / prod(df.shape)) * 100, 2), "%")
        print("- Total of full duplicates rows: ", df_rows_duplicates.shape[0])
        print("- Total of empty rows:           ", df.shape[0] - df.dropna(axis="rows", how="all").shape[0]) if df.dropna(axis="rows", how="all").shape[0] < df.shape[0] else \
                    print("- Total of empty rows:            0")
        print("- Total of empty columns:        ", len(empty_cols))
        print("  + The empty column is:         ", empty_cols) if len(empty_cols) == 1 else \
                    print("  + The empty column are:         ", empty_cols) if len(empty_cols) >= 1 else None
        
        print("\n- Type object and records by columns         (",memory_usage,")")
        print("--------------------------------------------------------------------------")
        print(df_resume.sort_values("Records", ascending=False))  
        
        # deleting dataframe to free memory
        del [df_resume]
        gc.collect()
        df_resume = pd.DataFrame()
    

<div style="background-color: #506AB9;" >
    <h2 style="margin: auto; padding: 20px; color:#fff; ">2. Importing files</h2>
</div>

<div style="background-color: #6D83C5;" >
    <h3 style="margin: auto; padding: 20px; color:#fff; ">2.1. Importing and preparing files</h3>
</div>

<div class="alert alert-block alert-info">
Reading data in <b>chunks of 1 million rows</b> at a time
</div>

In [4]:
start = time.time()
chunk = pd.read_csv("datasets/en.openfoodfacts.org.products.csv", chunksize=1000000, sep="\t", encoding="UTF-8")
data = pd.concat(chunk)
end = time.time()
print("Read csv with chunks: ",(end-start),"sec")

C:\Users\Samir\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (0,8,13,27,28,29,31,47,52,55,64) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\Samir\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (0,8,13,29,31,47,52,64,81,84) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Read csv with chunks:  100.97550177574158 sec


<div class="alert alert-block alert-info">
Making <b>the initial analysis</b>
</div>

<div class="alert alert-block alert-warning">
After analyzed the dataset, we can conclude the following:
<ol>
    <li>Almost 80% of the data present in data set are missing-values</li>
    <li>There are 5 empty columns that we can delete</li>
    <li>There is a lot of memory usage with this dataset</li>
</ol>
</div>

In [5]:
pd.set_option("display.max_rows", None) # Show full of rows
df_initial_analysis(data, "data")


Initial Analysis of data dataset
--------------------------------------------------------------------------
- Dataset shape:                  1760097 rows and 186 columns
- Total of NaN values:            260478039
- Percentage of NaN:              79.56 %
- Total of full duplicates rows:  1
- Total of empty rows:            0
- Total of empty columns:         5
  + The empty column are:          ['cities', 'allergens_en', 'no_nutriments', 'ingredients_from_palm_oil', 'ingredients_that_may_be_from_palm_oil']

- Type object and records by columns         ( memory usage: 2.4+ GB )
--------------------------------------------------------------------------
                                           Name     Type  Records
0                                          code   object  1760097
6                        last_modified_datetime   object  1760097
63                                    states_en   object  1760097
62                                  states_tags   object  1760097
61      

In [6]:
pd.set_option("display.max_rows", None) # reset max of showing rows

<div style="background-color: #506AB9;" >
    <h2 style="margin: auto; padding: 20px; color:#fff; ">2. Cleaning dataset</h2>
</div>

<div style="background-color: #6D83C5;" >
    <h3 style="margin: auto; padding: 20px; color:#fff; ">2.1. Deleting NaN columns and rows, and duplicated rows</h3>
</div>

In [7]:
data = data.dropna(axis="columns", how="all").dropna(axis="rows", how="all")

In [8]:
data = data.drop_duplicates()

<div class="alert alert-block alert-danger">
<b>I am here</b>
</div>

In [53]:
#Observation des colonnes qui contiennent le _t et le _datetime
print(data.iloc[:,data.columns.str.endswith('_t')].head())
print('\n', data.iloc[:,data.columns.str.endswith('_datetime')].head())

    created_t  last_modified_t
0  1529059080       1561463718
1  1539464774       1539464817
2  1574175736       1574175737
3  1619501895       1619501897
4  1444572561       1444659212

        created_datetime last_modified_datetime
0  2018-06-15T10:38:00Z   2019-06-25T11:55:18Z
1  2018-10-13T21:06:14Z   2018-10-13T21:06:57Z
2  2019-11-19T15:02:16Z   2019-11-19T15:02:17Z
3  2021-04-27T05:38:15Z   2021-04-27T05:38:17Z
4  2015-10-11T14:09:21Z   2015-10-12T14:13:32Z


In [ ]:
OBJECT_COLUMNS = [ 
    "code", "url", "creator", "created_datetime", "last_modified_datetime", "product_name", "abbreviated_product_name", "generic_name", 
    "quantity", "packaging", "packaging_tags", "packaging_text", "brands", "brands_tags", "categories", "categories_tags", "categories_en", 
    "origins", "origins_tags", "origins_en", "manufacturing_places", "manufacturing_places_tags", "labels", "labels_tags", "labels_en", 
    "emb_codes", "emb_codes_tags", "first_packaging_code_geo", "cities_tags", "purchase_places", "stores", "countries", "countries_tags", 
    "countries_en", "ingredients_text", "allergens", "traces", "traces_tags", "traces_en", "serving_size", "additives", "additives_tags", 
    "additives_en", "ingredients_from_palm_oil_tags", "ingredients_that_may_be_from_palm_oil_tags", "nutriscore_grade", "pnns_groups_1", 
    "pnns_groups_2", "states", "states_tags", "states_en", "brand_owner", "ecoscore_grade_fr", "main_category", "main_category_en", "image_url", 
    "image_small_url", "image_ingredients_url", "image_ingredients_small_url", "image_nutrition_url", "image_nutrition_small_url", "-butyric-acid_100g", 
    "-capric-acid_100g"    
]

In [10]:
for col in data.columns:
    print(col.dtypes)

AttributeError: 'str' object has no attribute 'dtypes'

In [43]:
OBJECT_COLUMNS, INT64_COLUMNS, FLOAT64_COLUMNS, CATEGORY_COLUMNS, BOOL_COLUMNS = ([] for i in range(5))

In [20]:
import numpy as np

In [44]:
for col in data.columns:
    if data[col].dtypes == "object" :
        OBJECT_COLUMNS.append(data[col].name)
    elif data[col].dtypes == "int64" :
        INT64_COLUMNS.append(data[col].name)
    elif data[col].dtypes == "float64" :
        FLOAT64_COLUMNS.append(data[col].name)
    elif data[col].dtypes == "category" :
        CATEGORY_COLUMNS.append(data[col].name)
    elif data[col].dtypes == "bool" :
        BOOL_COLUMNS.append(data[col].name)

In [52]:
data.head(1)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,brand_owner,ecoscore_score_fr,ecoscore_grade_fr,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,-soluble-fiber_100g,-insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,0000000000017,http://world-en.openfoodfacts.org/product/0000000000017/vitoria-crackers,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,Vitória crackers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed, en:nutrition-facts-completed, en:ingredients-to-be-completed, en:expiration-date-to-be-completed, en:packaging-code-to-be-completed, en:characteristics-to-be-completed, en:categories-to-be-completed, en:brands-to-be-completed, en:packaging-to-be-completed, en:quantity-to-be-completed, en:product-name-completed, en:photos-to-be-validated, en:photos-uploaded","en:to-be-completed,en:nutrition-facts-completed,en:ingredients-to-be-completed,en:

In [51]:
data[~data['code'].isna()]

MemoryError: 

MemoryError: 

In [26]:
data.dropna(axis="columns", how="all", inplace=True).dropna(axis="rows", how="all", inplace=True)

AttributeError: 'NoneType' object has no attribute 'dropna'

In [ ]:
data.drop_duplicates(inplace=True)

In [23]:
pd.set_option("display.max_rows", None) # Show full of rows
df_initial_analysis(data, "data")


Initial Analysis of data dataset
--------------------------------------------------------------------------
- Dataset shape:                  1760097 rows and 181 columns
- Total of NaN values:            251677554
- Percentage of NaN:              79.0 %
- Total of full duplicates rows:  1
- Total of empty rows:            0
- Total of empty columns:         0

- Type object and records by columns         ( memory usage: 2.4+ GB )
--------------------------------------------------------------------------
                                           Name     Type  Records
0                                          code   object  1760097
58                                    states_en   object  1760097
3                                     created_t    int64  1760097
4                              created_datetime   object  1760097
5                               last_modified_t    int64  1760097
6                        last_modified_datetime   object  1760097
57                        

In [24]:
pd.set_option("display.max_rows", None) # reset max of showing rows

In [19]:
data.tail(1)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,brand_owner,ecoscore_score_fr,ecoscore_grade_fr,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,-soluble-fiber_100g,-insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
1760096,999999999999999,http://world-en.openfoodfacts.org/product/999999999999999/signal-toothpaste,openfoodfacts-contributors,1587222660,2020-04-18T15:11:00Z,1605558295,2020-11-16T20:24:55Z,Signal Toothpaste,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Non food products, Open Beauty Facts, Toothpaste","en:non-food-products,en:open-beauty-facts,en:toothpaste","Non food products,Open Beauty Facts,Toothpaste",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed, en:nutrition-facts-to-be-completed, en:ingredients-to-be-completed, en:expiration-date-to-be-completed, en:packaging-code-to-be-completed, en:characteristics-to-be-completed, en:categories-completed, en:brands-to-be-completed, en:packaging-to-be-completed, en:quanti

In [20]:
data.describe()

,created_t,last_modified_t,cities,allergens_en,serving_quantity,no_nutriments,additives_n,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,nutriscore_score,nova_group,ecoscore_score_fr,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-caproic-acid_100g,-caprylic-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,-soluble-fiber_100g,-insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
count,1.760097e+06,1.760097e+06,0.0,0.0,4.633830e+05,0.0,687715.000000,687715.000000,0.0,687715.000000,0.0,655001.000000,601437.000000,421369.000000,1.274110e+05,1.343908e+06,1.397732e+06,975.000000,1.389669e+06,1.346364e+06,3.000000,3.000000,15.000000,7.000000,2.000000e+01,7.000000,78.000000,29.000000,3.000000e+00,8.000000,15.000000,10.000000,47404.000000,47404.000000,1981.000000,474.000000,117.000000,167.000000,508.000000,236.000000,73.000000,10.000000,5.000000,90.000000,53.000000,2.000000,1.500000e+01,8.000000,5.000000e+00,8.000000e+00,264553.000000,268350.000000,1.389200e+06,1.372059e+06,139.000000,77.000000,79.000000,769.000000,36.000000,89.000000,499.000000,3567.000000,467580.000000,3607.000000,3337.000000,1.390642e+06,43.000000,49.000000,23.000000,1.341585e+06,1.341581e+06,1.959300e+04,212152.000000,82.000000,9532.000000,3324.000000,1103.000000,219778.000000,23387.000000,22483.000000,23569.000000,15823.000000,10095.000000,8447.000000,1.257100e+04,1034.000000,5988.000000,118.000000,378.000000,91844.000000,685.000000,269712.000000,13664.000000,264442.000000,14579.000000,10096.000000,4237.000000,4002.000000,321.000000,2441.000000,184.000000,273.000000,2094.000000,425.00000,160.000000,187.000000,8792.000000,348.000000,11748.000000,309.000000,5904.000000,5.000000,455.000000,11632.000000,655006.000000,9.000000,4.000000,1.0,41.000000,1712.000000,32.000000,46.000000,20.000000
mean,1.551419e+09,1.586493e+09,NaN,NaN,2.397833e+16,NaN,2.045231,0.020606,NaN,0.069087,NaN,9.181360,3.431410,48.649775,5.231541e+37,6.535332e+06,4.768839e+36,361.496513,1.459468e+01,1.259576e+01,16.130000,43.133333,21.528927,34.779584,1.273718e+02,61.471143,10.715106,1.621922,1.388453e-03,2.753821,36.328254,0.284041,9.912199,5.892412,19.193757,2.876665,1.557723,0.869834,31.505225,4.464300,2.33578

In [ ]:
data.head(1)

In [7]:
nb_not_null = pd.DataFrame((~data.isna()).sum(axis =0), columns=['nb'])
nb_not_null.sort_values(by=['nb'], axis=0, ascending=True, inplace=True)
nb_not_null.T.head(150)

,cities,no_nutriments,ingredients_from_palm_oil,ingredients_that_may_be_from_palm_oil,allergens_en,water-hardness_100g,-elaidic-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-lignoceric-acid_100g,glycemic-index_100g,additives,-dihomo-gamma-linolenic-acid_100g,chlorophyl_100g,-erucic-acid_100g,-stearic-acid_100g,-myristic-acid_100g,-nervonic-acid_100g,-cerotic-acid_100g,-mead-acid_100g,nutrition-score-uk_100g,-melissic-acid_100g,-gamma-linolenic-acid_100g,-capric-acid_100g,-butyric-acid_100g,-gondoic-acid_100g,-montanic-acid_100g,-lauric-acid_100g,-palmitic-acid_100g,carnitine_100g,nucleotides_100g,-behenic-acid_100g,beta-glucan_100g,-maltose_100g,choline_100g,casein_100g,inositol_100g,serum-proteins_100g,-oleic-acid_100g,-arachidonic-acid_100g,-glucose_100g,-arachidic-acid_100g,-fructose_100g,beta-carotene_100g,-maltodextrins_100g,omega-9-fat_100g,-eicosapentaenoic-acid_100g,silica_100g,-sucrose_100g,taurine_100g,-docosahexaenoic-acid_100g,chromium_100g,ph_100g,-linoleic-acid_100g,molybdenum_100g,collagen-meat-protein-ratio_100g,fluoride_100g,fruits-vegetables-nuts-dried_100g,bicarbonate_100g,caffeine_100g,carbon-footprint_100g,-alpha-linolenic-acid_100g,starch_100g,omega-6-fat_100g,chloride_100g,-lactose_100g,energy-from-fat_100g,biotin_100g,vitamin-k_100g,phylloquinone_100g,omega-3-fat_100g,iodine_100g,selenium_100g,vitamin-e_100g,-insoluble-fiber_100g,abbreviated_product_name,polyols_100g,-soluble-fiber_100g,packaging_text,manganese_100g,copper_100g,cocoa_100g,pantothenic-acid_100g,folates_100g,fruits-vegetables-nuts_100g,vitamin-d_100g,vitamin-b9_100g,zinc_100g,carbon-footprint-from-meat-or-fish_100g,fruits-vegetables-nuts-estimate_100g,vitamin-b12_100g,phosphorus_100g,ingredients_from_palm_oil_tags,magnesium_100g,vitamin-b6_100g,alcohol_100g,vitamin-b2_100g,vitamin-b1_100g,vitamin-pp_100g,ingredients_that_may_be_from_palm_oil_tags,monounsaturated-fat_100g,polyunsaturated-fat_100g,first_packaging_code_geo,cities_tags,origins_tags,origins_en,origins,potassium_100g,traces,manufacturing_places_tags,manufacturing_places,emb_codes_tags,emb_codes,generic_name,traces_tags,traces_en,energy-kj_100g,purchase_places,allergens,vitamin-a_100g,stores,vitamin-c_100g,iron_100g,trans-fat_100g,cholesterol_100g,calcium_100g,packaging_tags,packaging,brand_owner,labels,labels_tags,labels_en,additives_tags,additives_en,ecoscore_score_fr,ecoscore_grade_fr,quantity,serving_quantity,fiber_100g,serving_size,nova_group,nutriscore_score,nutriscore_grade,nutrition-score-fr_100g,image_ingredients_url,image_ingredients_small_url,ingredients_text,ingredients_from_palm_oil_n,ingredients_that_may_be_from_palm_oil_n,additives_n,main_category_en,categories_en,categories_tags,main_category,categories,image_nutrition_small_url,image_nutrition_url,brands_tags,brands,image_url,image_small_url,sodium_100g,salt_100g,energy-kcal_100g,saturated-fat_100g,sugars_100g,carbohydrates_100g,fat_100g,proteins_100g,energy_100g,product_name,pnns_groups_1,countries_en,countries_tags,countries,pnns_groups_2,creator,url,last_modified_t,created_datetime,states,states_tags,states_en,last_modified_datetime,created_t,code
nb,0,0,0,0,0,1,2,3,3,3,4,4,5,5,5,7,7,8,8,8,9,10,10,10,14,15,15,15,20,20,23,29,32,36,41,43,46,49,53,73,77,78,79,82,89,90,117,118,139,160,167,184,187,236,273,309,321,348,378,425,455,474,499,508,685,769,975,1034,1103,1712,1981,2094,2441,3324,3337,3339,3567,3607,3722,4002,4237,5904,5988,8447,8792,9532,10095,10096,11632,11748,12571,13664,13994,14579,15823,19593,22483,23387,23569,40211,47404,47404,69575,74984,76186,76186,76304,91844,94000,112691,112742,112911,112942,113378,116895,116895,127411,151112,181644,212152,218680,219778,264442,264553,268350,269712,282838,282863,289516,388121,388140,388140,398474,398474,421369,421369,455823,463383,467580,468298,601437,655001,655001,655006,675186,675186,687713,687715,687715,687715,841866,841866,841866,841866,841867,858019,858019,918811,918867,1315404,1315404,1341581,1341585,1343908,1346364,1372059,1389200,1389669,1390642,1397732

In [46]:
pd.set_option("display.max_rows", None)
df_initial_analysis(data, "data")


Initial Analysis of data dataset
--------------------------------------------------------------------------
- Dataset shape:                  1760097 rows and 181 columns
- Total of NaN values:            251677554
- Percentage of NaN:              79.0 %
- Total of full duplicates rows:  1
- Total of empty rows:            0
- Total of empty columns:         0

- Type object and records by columns         ( memory usage: 2.4+ GB )
--------------------------------------------------------------------------
                                           Name    Type  Records
0                                          code  object  1760097
58                                    states_en  object  1760097
3                                     created_t  object  1760097
4                              created_datetime  object  1760097
5                               last_modified_t  object  1760097
6                        last_modified_datetime  object  1760097
57                               

In [13]:
pd.reset_option("display.max_columns") # reset max  of showing cols
pd.reset_option("max_colwidth") # reset full width of showing cols

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,abbreviated_product_name,generic_name,...,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
count,1760097,1760097,1760093,1760097,1760097,1760097,1760097,1682674,3339,113378,...,11632,655006,9,4,1,41,1712,32,46,20
unique,1760093,1760093,13454,1510529,1510529,1383412,1383412,1106740,3308,82578,...,2588,56,7,4,1,28,448,19,27,12
top,7340011495437,http://world-en.openfoodfacts.org/product/7340...,kiliweb,1587662527,2020-04-23T17:22:07Z,1614416585,2021-02-27T09:03:05Z,Aceite de oliva virgen extra,"6x27,5cl tourtel twi",Pâtes alimentaires de qualité supérieure,...,3580,14,22,48.7,9.1,0.069,0.0000083,4,0.02,0.0065
freq,2,2,989099,28,28,145,145,1252,7,283,...,329,34069,3,1,1,6,78,9,6,3


In [ ]:
# copying the dataset into new dataset and removing the NaN columns anr rows to work with the df
df_data_copy = df_data.dropna(axis="columns", how="all").dropna(axis="rows", how="all")

In [57]:
del [data]
gc.collect()
data = pd.DataFrame()
del [data]

NameError: name 'data' is not defined

In [10]:
pd.reset_option("display.max_rows") # reset width of showing rows
del [data]
gc.collect()
data = pd.DataFrame()
del [data]

<div class="alert alert-block alert-info">
Reading only 1000 rows <b>to know the dataset</b>
</div>

In [7]:
df_temp = pd.read_csv("datasets/en.openfoodfacts.org.products.csv", nrows=1000, sep="\t", encoding="UTF-8", dtype="unicode")

In [21]:
pd.set_option("display.max_columns", None)
df_temp.head(5)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,abbreviated_product_name,generic_name,quantity,packaging,packaging_tags,packaging_text,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,origins_en,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,serving_quantity,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutriscore_score,nutriscore_grade,nova_group,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,brand_owner,ecoscore_score_fr,ecoscore_grade_fr,main_category,main_category_en,image_url,image_small_url,image_ingredients_url,image_ingredients_small_url,image_nutrition_url,image_nutrition_small_url,energy-kj_100g,energy-kcal_100g,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,-soluble-fiber_100g,-insoluble-fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-dried_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,carbon-footprint-from-meat-or-fish_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1561463718,2019-06-25T11:55:18Z,Vitória crackers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",NaN,NaN,NaN,NaN,NaN,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,https://static.openfoodfacts.org/images/produc...,NaN,NaN,NaN,375,1569,NaN,7,3.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [30]:
pd.reset_option("display.max_columns") # reset width of showing cols

In [31]:
pd.set_option("display.max_rows", None)
df_temp.dtypes

code                                          object
url                                           object
creator                                       object
created_t                                     object
created_datetime                              object
last_modified_t                               object
last_modified_datetime                        object
product_name                                  object
abbreviated_product_name                      object
generic_name                                  object
quantity                                      object
packaging                                     object
packaging_tags                                object
packaging_text                                object
brands                                        object
brands_tags                                   object
categories                                    object
categories_tags                               object
categories_en                                 

In [32]:
pd.reset_option("display.max_rows") # reset width of showing rows

In [ ]:
<div class="alert alert-block alert-info">
<b>Nutri-Score data:</b>
    <ul>
      <li>Coffee</li>
      <li>Tea</li>
      <li>Milk</li>
    </ul>    
</div>

<div class="alert alert-block alert-warning">
<b>Example:</b> read data in chunks of 1 million rows at a time
</div>

<div class="alert alert-block alert-success">
<b>Up to you:</b> Use green boxes sparingly, and only for some specific 
purpose that the other boxes can't cover. For example, if you have a lot 
of related content to link to, maybe you decide to use green boxes for 
related links from each section of a notebook.
</div>

<div class="alert alert-block alert-danger">
<b>Just don't:</b> In general, avoid the red boxes. These should only be
used for actions that might cause data loss or another major issue.
</div>